# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid
from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## calculate_gini

In [2]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_periodic_gini_prod_ver_trench

In [3]:
def calculate_periodic_gini_prod_ver_trench(df, score_column, label_column, namecolumn, 
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None):
    """
    Calculate periodic Gini coefficients at multiple levels:
    - Overall
    - By model version
    - By all combinations of model version, trench category, loan type, and loan product type
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    model_version_column: (optional) name of column for model version (e.g., 'modelVersionId')
    trench_column: (optional) name of column for trench category (e.g., 'trenchCategory')
    loan_type_column: (optional) name of loan type column (e.g., 'loan_type')
    loan_product_type_column: (optional) name of loan product type column (e.g., 'loan_product_type')
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Add model version level segmentation
    if model_version_column:
        for model_version in sorted(df[model_version_column].dropna().unique()):
            model_df = df[df[model_version_column] == model_version]
            metadata = {model_version_column: model_version}
            datasets_to_process.append((f'ModelVersion_{model_version}', model_df, metadata))
    
    # Add all combinations of model_version, trench, loan_type, loan_product_type
    segment_columns = []
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    
    # Generate all combinations (only if we have multiple segment columns)
    if len(segment_columns) > 1:
        # Create combinations by iterating through all segment dimensions
        def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
            if current_filter is None:
                current_filter = {}
            
            if index >= len(segment_columns):
                # Apply all filters and create segment
                filtered_df = df
                for col, val in current_filter.items():
                    filtered_df = filtered_df[filtered_df[col] == val]
                
                if len(filtered_df) > 0:
                    yield (current_name.strip('_'), filtered_df, current_filter.copy())
                return
            
            seg_name, seg_col = segment_columns[index]
            for seg_value in sorted(df[seg_col].dropna().unique()):
                new_filter = current_filter.copy()
                new_filter[seg_col] = seg_value
                new_name = current_name + f'{seg_name}_{seg_value}_'
                
                yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
        
        for combo_name, combo_df, combo_metadata in generate_combinations(df, segment_columns):
            datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        if model_version_column and model_version_column in metadata:
            gini_results['model_version'] = metadata[model_version_column]
        else:
            gini_results['model_version'] = None
        
        if trench_column and trench_column in metadata:
            gini_results['trench_category'] = metadata[trench_column]
        else:
            gini_results['trench_category'] = None
        
        if loan_type_column and loan_type_column in metadata:
            gini_results['loan_type'] = metadata[loan_type_column]
        else:
            gini_results['loan_type'] = None
        
        if loan_product_type_column and loan_product_type_column in metadata:
            gini_results['loan_product_type'] = metadata[loan_product_type_column]
        else:
            gini_results['loan_product_type'] = None
        
        gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
        
        all_results.append(gini_results)
    
    # Combine all results
    final_results = pd.concat(all_results, ignore_index=True)
    
    return final_results


# Usage:
# gini_results = calculate_periodic_gini_prod_ver_trench(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type'
# )

In [4]:
# def calculate_periodic_gini_prod_ver_trench_md(df, score_column, label_column, namecolumn, 
#                                         model_version_column=None, trench_column=None, 
#                                         loan_type_column=None, loan_product_type_column=None,
#                                         account_id_column=None):
#     """
#     Calculate periodic Gini coefficients at multiple levels:
#     - Overall
#     - By model version
#     - By all combinations of model version, trench category, loan type, and loan product type
    
#     Parameters:
#     df: DataFrame with disbursement dates and score/label columns
#     score_column: name of the score column
#     label_column: name of the label column
#     namecolumn: name for the bad rate label
#     model_version_column: (optional) name of column for model version (e.g., 'modelVersionId')
#     trench_column: (optional) name of column for trench category (e.g., 'trenchCategory')
#     loan_type_column: (optional) name of loan type column (e.g., 'loan_type')
#     loan_product_type_column: (optional) name of loan product type column (e.g., 'loan_product_type')
#     account_id_column: (optional) name of column for distinct account IDs (e.g., 'digitalLoanAccountId')
#     """
#     # Input validation
#     required_columns = ['disbursementdate', score_column, label_column]
#     if not all(col in df.columns for col in required_columns):
#         raise ValueError(f"Missing required columns. Need: {required_columns}")
    
#     optional_columns = {
#         'model_version': model_version_column,
#         'trench': trench_column,
#         'loan_type': loan_type_column,
#         'loan_product_type': loan_product_type_column,
#         'account_id': account_id_column
#     }
    
#     for col_name, col in optional_columns.items():
#         if col and col not in df.columns:
#             raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
#     # Create a copy to avoid modifying original dataframe
#     df = df.copy()
    
#     # Ensure date is datetime type
#     df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
#     # Ensure score and label columns are numeric
#     df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
#     df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
#     # Drop rows with invalid values
#     df = df.dropna(subset=[score_column, label_column])
    
#     # Define list of datasets to process
#     datasets_to_process = [('Overall', df, {})]
    
#     # Add model version level segmentation
#     if model_version_column:
#         for model_version in sorted(df[model_version_column].dropna().unique()):
#             model_df = df[df[model_version_column] == model_version]
#             metadata = {model_version_column: model_version}
#             datasets_to_process.append((f'ModelVersion_{model_version}', model_df, metadata))
    
#     # Add all combinations of model_version, trench, loan_type, loan_product_type
#     segment_columns = []
#     if model_version_column:
#         segment_columns.append(('ModelVersion', model_version_column))
#     if trench_column:
#         segment_columns.append(('Trench', trench_column))
#     if loan_type_column:
#         segment_columns.append(('LoanType', loan_type_column))
#     if loan_product_type_column:
#         segment_columns.append(('ProductType', loan_product_type_column))
    
#     # Generate all combinations (only if we have multiple segment columns)
#     if len(segment_columns) > 1:
#         # Create combinations by iterating through all segment dimensions
#         def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
#             if current_filter is None:
#                 current_filter = {}
            
#             if index >= len(segment_columns):
#                 # Apply all filters and create segment
#                 filtered_df = df
#                 for col, val in current_filter.items():
#                     filtered_df = filtered_df[filtered_df[col] == val]
                
#                 if len(filtered_df) > 0:
#                     yield (current_name.strip('_'), filtered_df, current_filter.copy())
#                 return
            
#             seg_name, seg_col = segment_columns[index]
#             for seg_value in sorted(df[seg_col].dropna().unique()):
#                 new_filter = current_filter.copy()
#                 new_filter[seg_col] = seg_value
#                 new_name = current_name + f'{seg_name}_{seg_value}_'
                
#                 yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
        
#         for combo_name, combo_df, combo_metadata in generate_combinations(df, segment_columns):
#             datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
#     all_results = []
    
#     # Process each dataset
#     for dataset_name, dataset_df, metadata in datasets_to_process:
#         # Calculate weekly Gini
#         dataset_df_copy = dataset_df.copy()
#         dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
#         weekly_gini = dataset_df_copy.groupby('week').apply(
#             lambda x: calculate_gini(x[score_column], x[label_column])
#             if len(x) >= 10 else np.nan
#         ).reset_index(name='gini')
#         weekly_gini['period'] = 'Week'
#         weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
#         weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
#         # Add distinct account count for weekly
#         if account_id_column:
#             weekly_gini['distinct_accounts'] = dataset_df_copy.groupby('week')[account_id_column].nunique().values
        
#         weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
        
#         # Calculate monthly Gini
#         dataset_df_copy = dataset_df.copy()
#         dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
#         monthly_gini = dataset_df_copy.groupby('month').apply(
#             lambda x: calculate_gini(x[score_column], x[label_column])
#             if len(x) >= 20 else np.nan
#         ).reset_index(name='gini')
#         monthly_gini['period'] = 'Month'
#         monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
#         monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
#         # Add distinct account count for monthly
#         if account_id_column:
#             monthly_gini['distinct_accounts'] = dataset_df_copy.groupby('month')[account_id_column].nunique().values
        
#         monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
        
#         # Combine results for this dataset
#         gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
#         gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
#         # Add metadata columns
#         gini_results['Model_Name'] = score_column
#         gini_results['bad_rate'] = namecolumn
#         gini_results['segment_type'] = dataset_name
        
#         # Add individual segment components
#         if model_version_column and model_version_column in metadata:
#             gini_results['model_version'] = metadata[model_version_column]
#         else:
#             gini_results['model_version'] = None
        
#         if trench_column and trench_column in metadata:
#             gini_results['trench_category'] = metadata[trench_column]
#         else:
#             gini_results['trench_category'] = None
        
#         if loan_type_column and loan_type_column in metadata:
#             gini_results['loan_type'] = metadata[loan_type_column]
#         else:
#             gini_results['loan_type'] = None
        
#         if loan_product_type_column and loan_product_type_column in metadata:
#             gini_results['loan_product_type'] = metadata[loan_product_type_column]
#         else:
#             gini_results['loan_product_type'] = None
        
#         gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
        
#         all_results.append(gini_results)
    
#     # Combine all results
#     final_results = pd.concat(all_results, ignore_index=True)
    
#     return final_results


# # Usage:
# # gini_results = calculate_periodic_gini_prod_ver_trench(
# #     df_concat, 
# #     'Alpha_cic_sil_score', 
# #     'deffpd0', 
# #     'FPD0',
# #     model_version_column='modelVersionId',
# #     trench_column='trenchCategory',
# #     loan_type_column='loan_type',
# #     loan_product_type_column='loan_product_type',
# #     account_id_column='digitalLoanAccountId'
# # )

In [5]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """Calculate Gini coefficient"""
    n = len(scores)
    if n < 2:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * np.sum(sorted_labels))
    return gini


def calculate_periodic_gini_prod_ver_trench_md(df, score_column, label_column, namecolumn, 
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients at multiple levels:
    - Overall (entire dataset)
    - By each individual segment (model version, trench, loan type, loan product type)
    - By all possible combinations of segments
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    
    # Generate all possible combinations of segment columns (from single to all)
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            # Create combinations by iterating through all segment dimensions
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    # Apply all filters and create segment
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
        # Add distinct account count for weekly
        if account_id_column:
            weekly_account_counts = dataset_df_copy.groupby('week')[account_id_column].nunique().reset_index()
            weekly_account_counts.columns = ['week', 'distinct_accounts']
            weekly_gini = weekly_gini.merge(weekly_account_counts, on='week', how='left')
        
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period'] + (['distinct_accounts'] if account_id_column else [])]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
        # Add distinct account count for monthly
        if account_id_column:
            monthly_account_counts = dataset_df_copy.groupby('month')[account_id_column].nunique().reset_index()
            monthly_account_counts.columns = ['month', 'distinct_accounts']
            monthly_gini = monthly_gini.merge(monthly_account_counts, on='month', how='left')
        
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period'] + (['distinct_accounts'] if account_id_column else [])]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        if model_version_column and model_version_column in metadata:
            gini_results['model_version'] = metadata[model_version_column]
        else:
            gini_results['model_version'] = None
        
        if trench_column and trench_column in metadata:
            gini_results['trench_category'] = metadata[trench_column]
        else:
            gini_results['trench_category'] = None
        
        if loan_type_column and loan_type_column in metadata:
            gini_results['loan_type'] = metadata[loan_type_column]
        else:
            gini_results['loan_type'] = None
        
        if loan_product_type_column and loan_product_type_column in metadata:
            gini_results['loan_product_type'] = metadata[loan_product_type_column]
        else:
            gini_results['loan_product_type'] = None
        
        gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
        
        all_results.append(gini_results)
    
    # Combine all results
    final_results = pd.concat(all_results, ignore_index=True)
    
    return final_results


# Usage:
# gini_results = calculate_periodic_gini_prod_ver_trench_md(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     account_id_column='digitalLoanAccountId'
# )

In [6]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """Calculate Gini coefficient"""
    n = len(scores)
    if n < 2:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * np.sum(sorted_labels))
    return gini


def calculate_periodic_gini_prod_ver_trench_md_col(df, score_column, label_column, namecolumn, 
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients at multiple levels:
    - Overall (entire dataset)
    - By each individual segment (model version, trench, loan type, loan product type)
    - By all possible combinations of segments
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    
    # Generate all possible combinations of segment columns (from single to all)
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            # Create combinations by iterating through all segment dimensions
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    # Apply all filters and create segment
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
        # Add distinct account count for weekly
        if account_id_column:
            weekly_account_counts = dataset_df_copy.groupby('week')[account_id_column].nunique().reset_index()
            weekly_account_counts.columns = ['week', 'distinct_accounts']
            weekly_gini = weekly_gini.merge(weekly_account_counts, on='week', how='left')
        
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period'] + (['distinct_accounts'] if account_id_column else [])]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
        # Add distinct account count for monthly
        if account_id_column:
            monthly_account_counts = dataset_df_copy.groupby('month')[account_id_column].nunique().reset_index()
            monthly_account_counts.columns = ['month', 'distinct_accounts']
            monthly_gini = monthly_gini.merge(monthly_account_counts, on='month', how='left')
        
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period'] + (['distinct_accounts'] if account_id_column else [])]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        if model_version_column and model_version_column in metadata:
            gini_results['model_version'] = metadata[model_version_column]
        else:
            gini_results['model_version'] = None
        
        if trench_column and trench_column in metadata:
            gini_results['trench_category'] = metadata[trench_column]
        else:
            gini_results['trench_category'] = None
        
        if loan_type_column and loan_type_column in metadata:
            gini_results['loan_type'] = metadata[loan_type_column]
        else:
            gini_results['loan_type'] = None
        
        if loan_product_type_column and loan_product_type_column in metadata:
            gini_results['loan_product_type'] = metadata[loan_product_type_column]
        else:
            gini_results['loan_product_type'] = None
        
        # Rename gini column with segment type included
        gini_col_name = f'{score_column}_{namecolumn}_gini_{dataset_name}'
        gini_results.rename(columns={'gini': gini_col_name}, inplace=True)
        
        all_results.append(gini_results)
    
    # Combine all results by merging on time periods
    final_results = all_results[0][['start_date', 'end_date', 'period', 'Model_Name', 'bad_rate']].copy()
    
    if account_id_column:
        final_results['distinct_accounts'] = all_results[0]['distinct_accounts']
    
    for result in all_results:
        gini_col = [col for col in result.columns if 'gini' in col][0]
        final_results = final_results.merge(
            result[['start_date', 'end_date', 'period', gini_col]],
            on=['start_date', 'end_date', 'period'],
            how='outer'
        )
    
    # Fill NaN values and sort by date
    final_results = final_results.sort_values(by='start_date').reset_index(drop=True)
    
    return final_results


# Usage:
# gini_results = calculate_periodic_gini_prod_ver_trench_md(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     account_id_column='digitalLoanAccountId'
# )

# SIL

## Alpha_cic_sil_score

## FPD0

## Test

In [17]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2757224,7d9e9b1b-9704-48dc-b5d4-e1f00d6367bc,60827572240023,cic_model_sil,0.06701889001773045,2025-06-27 14:12:43,2025-06-27,2025-06,Test,0,1,SIL-Instore,Appliance,v1,ALL
1,3064349,1b14fc74-e072-41c0-8cb4-e0bc76641324,60830643490023,cic_model_sil,0.13122331884664834,2025-06-27 14:47:01,2025-06-27,2025-06,Test,0,1,SIL-Instore,Appliance,v1,ALL
2,3280410,57332877-b6e1-4b70-9cca-c9cc826a440e,60832804100019,cic_model_sil,0.08544301996560745,2025-06-27 14:19:24,2025-06-27,2025-06,Test,0,1,SIL-Instore,Appliance,v1,ALL
3,3522986,b73bc9cc-6670-4bc9-97b2-1360fd36e885,60835229860011,cic_model_sil,0.10229841609867538,2025-06-27 13:52:00,2025-06-27,2025-06,Test,0,1,SIL-Instore,Appliance,v1,ALL
4,3522992,528b20b5-72d2-4061-92d0-7fed6709294b,60835229920018,cic_model_sil,0.1656376977469904,2025-06-27 13:56:16,2025-06-27,2025-06,Test,0,1,SIL-Instore,Appliance,v1,ALL


In [ ]:
df1 = dfd.copy()

In [ ]:
df1['modelVersionId'].value_counts()

modelVersionId
v1    46447
Name: count, dtype: int64

## Train

In [21]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    deffpd0,
  flg_mature_fpd0,
   loanmaster.new_loan_type,
          case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId,
    trenchCategory,
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2253535,92b5f956-a31f-4a94-8b94-f3f29e086c04,60822535350011,Alpha - CIC-SIL-Model,0.122954,2023-09-30 11:34:38,2023-09-30,2023-09,Train,0,1,SIL-Instore,Appliance,v1,ALL
1,2178789,e8ea5e00-49b7-44d1-a5e3-d14b17727401,60821787890015,Alpha - CIC-SIL-Model,0.125567,2023-08-08 10:36:56,2023-08-08,2023-08,Train,0,1,SIL-Instore,Appliance,v1,ALL
2,2247864,134235a5-7554-4129-ae9c-0fa5c2c874f1,60822478640016,Alpha - CIC-SIL-Model,0.100982,2023-09-25 18:01:27,2023-09-25,2023-09,Train,0,1,SIL-Instore,Appliance,v1,ALL
3,2403204,d2d33414-7031-4efc-ab1f-4d8bdf2a36e2,60824032040012,Alpha - CIC-SIL-Model,0.124636,2024-02-17 19:22:18,2024-02-17,2024-02,Train,0,1,SIL-Instore,Appliance,v1,ALL
4,2393961,5fa3245c-7d63-4e8e-83fa-de1f878350e6,60823939610011,Alpha - CIC-SIL-Model,0.112962,2024-02-08 17:43:55,2024-02-13,2024-02,Train,1,1,SIL-Instore,Appliance,v1,ALL


In [22]:
df2 = dfd.copy()

In [23]:
df2[['modelVersionId', 'modelDisplayName']].value_counts()    

modelVersionId  modelDisplayName     
v2              cic_model_sil            159354
v1              Alpha - CIC-SIL-Model    132827
Name: count, dtype: int64

In [24]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339659 entries, 0 to 339658
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             339659 non-null  object        
 1   digitalLoanAccountId   339659 non-null  object        
 2   loanAccountNumber      339659 non-null  object        
 3   modelDisplayName       339659 non-null  object        
 4   Alpha_cic_sil_score    339659 non-null  object        
 5   appln_submit_datetime  339659 non-null  datetime64[us]
 6   disbursementdate       339659 non-null  dbdate        
 7   disbursement_month     47478 non-null   object        
 8   disbmonth              47478 non-null   dbdate        
 9   disbursedweek          47478 non-null   dbdate        
 10  Data_selection         339659 non-null  object        
 11  deffpd0                339659 non-null  Int64         
 12  flg_mature_fpd0        339659 non-null  Int6

In [26]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')

In [27]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()

Application_month
2023-01      185
2023-02      195
2023-03      231
2023-04      282
2023-05      290
2023-06      307
2023-07      320
2023-08      711
2023-09     1187
2023-10     1384
2023-11     1877
2023-12     3409
2024-01     1811
2024-02     1418
2024-03     2071
2024-04     3810
2024-05     5053
2024-06     8666
2024-07     8484
2024-08    11546
2024-09    22523
2024-10    21417
2024-11    23861
2024-12    44300
2025-01    20734
2025-02    18164
2025-03    14943
2025-04    13012
2025-05    13424
2025-06    12174
2025-07    11867
2025-08    11953
2025-09    10570
2025-11        2
NaN        47478
Name: count, dtype: int64

In [ ]:
# # gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffpd0', 'FPD0')

# # gini_results = calculate_periodic_gini_producttype(
# #     df_concat, 
# #     'Alpha_cic_sil_score', 
# #     'deffpd0', 
# #     'FPD0',
# #     product_column='new_loan_type'
# # )

# gini_results = calculate_periodic_gini_prod_ver_trench(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     product_column='new_loan_type'
# )

In [28]:
f0 = df_concat.copy()
f0.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,disbursement_month,disbmonth,disbursedweek,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,Application_month
0,3357622,4e0f45b7-f555-48a5-a204-8fe496cc1c36,60833576220018,cic_model_sil,0.214982,2025-04-03 17:56:15,2025-04-03,2025-04,2025-04-01,2025-03-31,Test,0,1,SIL-Instore,v1,ALL,Appliance,NaN
1,3704175,c5d55c81-f42e-421d-bb31-d9199dbc290c,60837041750013,cic_model_sil,0.147605,2025-09-25 12:39:02,2025-09-25,2025-09,2025-09-01,2025-09-22,Test,0,1,SIL Competitor,v1,ALL,Mall,NaN
2,3703835,7c4eb0fe-3d0e-4292-8e24-e914da2b2eff,60837038350012,cic_model_sil,0.123423,2025-09-25 09:51:28,2025-09-25,2025-09,2025-09-01,2025-09-22,Test,0,1,SIL-Instore,v1,ALL,Appliance,NaN
3,3703951,a9a2df60-6d23-44e8-8e3d-582c75ac7607,60837039510015,cic_model_sil,0.098974,2025-09-25 10:59:57,2025-09-25,2025-09,2025-09-01,2025-09-22,Test,0,1,SIL-Instore,v1,ALL,Appliance,NaN
4,3704066,b2ecd9a2-520e-4397-8e7e-055f5474a426,60837040660019,cic_model_sil,0.124237,2025-09-25 11:54:15,2025-09-25,2025-09,2025-09-01,2025-09-22,Test,0,1,SIL-Instore,v1,ALL,Appliance,NaN


In [29]:
f0.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'modelDisplayName', 'Alpha_cic_sil_score', 'appln_submit_datetime',
       'disbursementdate', 'disbursement_month', 'disbmonth', 'disbursedweek',
       'Data_selection', 'deffpd0', 'flg_mature_fpd0', 'new_loan_type',
       'modelVersionId', 'trenchCategory', 'loan_product_type',
       'Application_month'],
      dtype='object')

## FPD10

## Test

In [30]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
         case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd10 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3529497,07c9b0ce-2976-49de-983f-60979714f4f6,60835294970016,0.1015182478135047,2025-06-30 13:24:36,2025-06-30,2025-06,Test,0,1,SIL-Instore,Appliance,v1,ALL
1,3529490,0bed6ad2-127e-4962-9217-3c4790ef3b6e,60835294900019,0.03997691952577809,2025-06-30 13:30:20,2025-06-30,2025-06,Test,0,1,SIL Competitor,Mall,v1,ALL
2,3529456,18f3ca19-648c-4a92-9c2b-355aa264774d,60835294560017,0.12277073399662226,2025-06-30 13:11:35,2025-06-30,2025-06,Test,0,1,SIL ZERO,Appliance,v1,ALL
3,3529582,1b02039a-3c20-411e-94dd-78e33d7a836d,60835295820012,0.10496616293231648,2025-06-30 14:10:16,2025-06-30,2025-06,Test,0,1,SIL Competitor,Appliance,v1,ALL
4,3529489,1dad6873-f647-4364-889b-0e4b8a1ffb67,60835294890016,0.18794747755934035,2025-06-30 13:32:58,2025-06-30,2025-06,Test,1,1,SIL-Instore,Appliance,v1,ALL


In [31]:
df1 = dfd.copy()

## Train

In [32]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
        case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
     ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffpd10,
  flg_mature_fpd10,
  loanmaster.new_loan_type,
         case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd10 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2390720,7071e7b7-655c-4dd7-9f1e-537f64220a41,60823907200014,0.123149,2024-02-05 13:55:42,2024-02-05,2024-02,Train,0,1,SIL-Instore,Appliance,v1,ALL
1,2409058,564ebe1d-6057-48e0-9798-3b2d1a3d2407,60824090580014,0.199273,2024-02-23 16:41:32,2024-02-23,2024-02,Train,1,1,SIL-Instore,Appliance,v1,ALL
2,2401124,0a9718f8-c0be-4261-8758-464f49719401,60824011240015,0.125567,2024-02-15 18:35:33,2024-02-15,2024-02,Train,0,1,SIL-Instore,Appliance,v1,ALL
3,2205372,af03e82f-ba69-4df2-8ab9-42cae55cd768,60822053720013,0.125483,2023-08-24 14:29:19,2023-08-24,2023-08,Train,0,1,SIL-Instore,Appliance,v1,ALL
4,2259980,a92d5302-4e1c-4b33-8d1e-0cd7d813dd56,60822599800012,0.209609,2023-10-04 19:01:58,2023-10-07,2023-10,Train,0,1,SIL-Instore,Appliance,v1,ALL


In [33]:
df2 = dfd.copy()

In [34]:
df2['modelVersionId'].value_counts()

modelVersionId
v2    159354
v1    132827
Name: count, dtype: int64

In [35]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335159 entries, 0 to 335158
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             335159 non-null  object        
 1   digitalLoanAccountId   335159 non-null  object        
 2   loanAccountNumber      335159 non-null  object        
 3   Alpha_cic_sil_score    335159 non-null  object        
 4   appln_submit_datetime  335159 non-null  datetime64[us]
 5   disbursementdate       335159 non-null  dbdate        
 6   Application_month      335159 non-null  object        
 7   Data_selection         335159 non-null  object        
 8   deffpd10               335159 non-null  Int64         
 9   flg_mature_fpd10       335159 non-null  Int64         
 10  new_loan_type          335159 non-null  object        
 11  loan_product_type      335159 non-null  object        
 12  modelVersionId         335159 non-null  obje

In [36]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffpd10', 'FPD10')
# gini_results = calculate_periodic_gini_producttype(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd10', 
#     'FPD10',
#     product_column='new_loan_type'
# )

# gini_results = calculate_periodic_gini_prod_ver_trench(
#     df_concat,
#     'Alpha_cic_sil_score', 
#     'deffpd10', 
#     'FPD10',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     product_column='new_loan_type'
# )

In [39]:
f1=df_concat.copy()


In [40]:
f1.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'Alpha_cic_sil_score', 'appln_submit_datetime', 'disbursementdate',
       'Application_month', 'Data_selection', 'deffpd10', 'flg_mature_fpd10',
       'new_loan_type', 'loan_product_type', 'modelVersionId',
       'trenchCategory'],
      dtype='object')

## FPD30

## Test

In [41]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
         case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3115385,1517dd71-382e-4542-a255-b095d5dcdbe5,60831153850014,0.08242221512070065,2025-09-08 14:08:18,2025-09-08,2025-09,Test,0,1,SIL Competitor,Appliance,v1,ALL
1,3672505,18fa5990-016f-470d-8166-c79c16bdf1e7,60836725050012,0.1268715555250324,2025-09-08 15:44:19,2025-09-08,2025-09,Test,0,1,SIL-Instore,Appliance,v1,ALL
2,3672335,1d8eab50-7447-4a38-8526-e65608035862,60836723350012,0.07862029959238276,2025-09-08 14:46:02,2025-09-08,2025-09,Test,0,1,SIL-Instore,Mall,v1,ALL
3,3672361,219a7bc2-fdb0-4d7d-8584-e50263fa7e74,60836723610014,0.05457006515040178,2025-09-08 14:43:01,2025-09-08,2025-09,Test,0,1,SIL Competitor,Appliance,v1,ALL
4,3672397,2687223c-4803-4c2c-a4df-3ac77727595e,60836723970016,0.08062146089573777,2025-09-08 15:01:29,2025-09-08,2025-09,Test,0,1,SIL Competitor,Appliance,v1,ALL


In [42]:
df1 = dfd.copy()

## Train

In [43]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
    ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffpd30,
  flg_mature_fpd30,
  loanmaster.new_loan_type,
         case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2114137,1084c288-08bc-44f5-bf1d-3b4d2f229df9,60821141370019,0.081278,2023-06-30 18:34:07,2023-06-30,2023-06,Train,0,1,SIL-Instore,Appliance,v1,ALL
1,2282761,51eb34a6-3347-48d1-a8e3-d80454528e47,60822827610011,0.199273,2023-10-24 10:40:57,2023-10-24,2023-10,Train,0,1,SIL-Instore,Appliance,v1,ALL
2,2389931,029e868a-6f33-482c-959d-4c7df2172a28,60823899310014,0.214070,2024-02-04 17:43:12,2024-02-04,2024-02,Train,1,1,SIL-Instore,Appliance,v1,ALL
3,2264925,e071b474-cc6e-4ebd-81ac-076207a42184,60822649250011,0.219905,2023-10-08 15:00:19,2023-10-08,2023-10,Train,0,1,SIL-Instore,Appliance,v1,ALL
4,2254831,0dc2d368-89eb-47b5-aa74-d0255385314d,60822548310011,0.125541,2023-10-01 11:10:54,2023-10-01,2023-10,Train,0,1,SIL-Instore,Appliance,v1,ALL


In [44]:
df2 = dfd.copy()

In [45]:
df2['modelVersionId'].value_counts()

modelVersionId
v2    159334
v1    132827
Name: count, dtype: int64

In [46]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329093 entries, 0 to 329092
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             329093 non-null  object        
 1   digitalLoanAccountId   329093 non-null  object        
 2   loanAccountNumber      329093 non-null  object        
 3   Alpha_cic_sil_score    329093 non-null  object        
 4   appln_submit_datetime  329093 non-null  datetime64[us]
 5   disbursementdate       329093 non-null  dbdate        
 6   Application_month      329093 non-null  object        
 7   Data_selection         329093 non-null  object        
 8   deffpd30               329093 non-null  Int64         
 9   flg_mature_fpd30       329093 non-null  Int64         
 10  new_loan_type          329093 non-null  object        
 11  loan_product_type      329093 non-null  object        
 12  modelVersionId         329093 non-null  obje

In [47]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffpd30', 'FPD30')


# gini_results = calculate_periodic_gini_producttype(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd30', 
#     'FPD30',
#     product_column='new_loan_type'
# )

# gini_results = calculate_periodic_gini_prod_ver_trench(
#     df_concat,
#     'Alpha_cic_sil_score', 
#     'deffpd30', 
#     'FPD30',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     product_column='new_loan_type'
# )

In [49]:
f2=df_concat.copy()


In [50]:
f2.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'Alpha_cic_sil_score', 'appln_submit_datetime', 'disbursementdate',
       'Application_month', 'Data_selection', 'deffpd30', 'flg_mature_fpd30',
       'new_loan_type', 'loan_product_type', 'modelVersionId',
       'trenchCategory'],
      dtype='object')

## FSPD30

## Test

In [51]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
    ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
         case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fspd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3347246,9b806dc0-aab6-4e4e-ad83-998b103b6ebd,60833472460019,0.12675888493226176,2025-03-29 15:51:41,2025-03-29,2025-03,Test,0,1,SIL-Instore,Appliance,v1,ALL
1,3659707,8d9207d2-4692-499a-b5ed-fef47893ec9d,60836597070018,0.08051806694960593,2025-09-02 16:20:09,2025-09-02,2025-09,Test,0,1,SIL-Instore,Appliance,v1,ALL
2,3477793,a707bb20-73f3-45bf-a4fe-94822187d926,60834777930012,0.12566888018908812,2025-06-04 16:42:43,2025-06-04,2025-06,Test,0,1,SIL ZERO,Appliance,v1,ALL
3,3104667,88942fa8-ea3f-4a14-8493-a516c0a40df3,60831046670021,0.15451064290610725,2025-05-03 18:28:15,2025-05-03,2025-05,Test,0,1,SIL-Instore,Appliance,v1,ALL
4,3479987,2d7c68f3-c78a-45f8-b60b-bf4c1f129be3,60834799870018,0.08907449153597956,2025-06-05 19:53:20,2025-06-05,2025-06,Test,0,1,SIL-Instore,Appliance,v1,ALL


In [52]:
df1 = dfd.copy()

## Train

In [53]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
        case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
    ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffspd30,
  flg_mature_fspd_30,
  loanmaster.new_loan_type,
         case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fspd_30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2005234,fc1f500a-aeeb-4db5-bb1f-e14e610b9811,60820052340012,0.126860,2023-04-20 12:15:25,2023-04-20,2023-04,Train,0,1,SIL-Instore,Appliance,v1,ALL
1,2285214,2176d82e-dac7-4b5c-823f-10657381f086,60822852140011,0.109207,2023-10-26 19:00:12,2023-10-26,2023-10,Train,0,1,SIL-Instore,Appliance,v1,ALL
2,2175407,8307c082-dff3-4b27-b31a-5e6d18975639,60821754070016,0.077753,2023-08-06 11:43:39,2023-08-06,2023-08,Train,0,1,SIL-Instore,Appliance,v1,ALL
3,2264991,b51a2664-6373-470e-aa81-574639cb9acf,60822649910011,0.121799,2023-10-08 15:28:12,2023-10-08,2023-10,Train,0,1,SIL-Instore,Appliance,v1,ALL
4,3657502,b49494b9-e889-46f8-8f36-34e287c37ad8,60836575020019,0.344859,2025-09-01 16:37:46,2025-09-01,2025-09,Train,0,1,SIL ZERO,Appliance,v2,Trench 1


In [54]:
df2 = dfd.copy()

In [55]:
df2['modelVersionId'].value_counts()

modelVersionId
v2    148431
v1    132827
Name: count, dtype: int64

In [56]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308945 entries, 0 to 308944
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             308945 non-null  object        
 1   digitalLoanAccountId   308945 non-null  object        
 2   loanAccountNumber      308945 non-null  object        
 3   Alpha_cic_sil_score    308945 non-null  object        
 4   appln_submit_datetime  308945 non-null  datetime64[us]
 5   disbursementdate       308945 non-null  dbdate        
 6   Application_month      308945 non-null  object        
 7   Data_selection         308945 non-null  object        
 8   deffspd30              308945 non-null  Int64         
 9   flg_mature_fspd_30     308945 non-null  Int64         
 10  new_loan_type          308945 non-null  object        
 11  loan_product_type      308945 non-null  object        
 12  modelVersionId         308945 non-null  obje

In [57]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffspd30', 'FSPD30')
# gini_results = calculate_periodic_gini_producttype(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffspd30', 
#     'FSPD30',
#     product_column='new_loan_type'
# )

# gini_results = calculate_periodic_gini_prod_ver_trench(
#     df_concat,
#     'Alpha_cic_sil_score', 
#     'deffspd30', 
#     'FSPD30',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     product_column='new_loan_type'
# )

In [58]:
f3=df_concat.copy()

In [59]:
f3.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'Alpha_cic_sil_score', 'appln_submit_datetime', 'disbursementdate',
       'Application_month', 'Data_selection', 'deffspd30',
       'flg_mature_fspd_30', 'new_loan_type', 'loan_product_type',
       'modelVersionId', 'trenchCategory'],
      dtype='object')

## FSTPD30

## Test

In [60]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
    ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
         case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fstpd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3418475,058288be-7950-441f-943c-0652f25ab88b,60834184750015,0.08632429485335606,2025-05-04 10:18:56,2025-05-04,2025-05,Test,0,1,SIL ZERO,Appliance,v1,ALL
1,3418539,eb92cc66-336f-4e6b-b42c-828e8425e272,60834185390014,0.09656231650293898,2025-05-04 10:29:14,2025-05-04,2025-05,Test,0,1,SIL-Instore,Mall,v1,ALL
2,3418607,dd5efe11-d2ad-43ae-946c-e23dd7b6d36e,60834186070019,0.09897421800534903,2025-05-04 10:54:26,2025-05-04,2025-05,Test,0,1,SIL-Instore,Appliance,v1,ALL
3,3418436,996b8104-9db8-45cf-b491-ebe07b97217f,60834184360018,0.04943166238818924,2025-05-04 09:38:43,2025-05-04,2025-05,Test,0,1,SIL-Instore,Appliance,v1,ALL
4,3418626,9911a87c-4305-4036-982f-6a52133df3f2,60834186260012,0.17997236333844285,2025-05-04 11:06:10,2025-05-04,2025-05,Test,0,1,SIL-Instore,Appliance,v1,ALL


In [61]:
df1 = dfd.copy()

## Train

In [62]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
        case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
    ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffstpd30,
  flg_mature_fstpd_30,
  loanmaster.new_loan_type,
         case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fstpd_30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2250600,0f9be1d9-2bdb-4bda-847d-528a0d5c729b,60822506000011,0.140354,2023-09-27 19:24:53,2023-09-27,2023-09,Train,0,1,SIL-Instore,Appliance,v1,ALL
1,3591392,32c0f9bb-93f0-4c05-ac53-47099c2f5bf9,60835913920011,0.120998,2025-08-01 09:59:53,2025-08-01,2025-08,Train,0,1,SIL-Instore,Appliance,v2,Trench 1
2,3594530,84597bbd-70ec-40e1-8507-47f22b491622,60835945300015,0.448600,2025-08-02 17:24:53,2025-08-02,2025-08,Train,0,1,SIL-Instore,Appliance,v2,Trench 1
3,3593500,08cec71c-a100-4068-bb90-91e9ef2aef78,60835935000015,0.448600,2025-08-02 11:31:48,2025-08-02,2025-08,Train,1,1,SIL Competitor,Appliance,v2,Trench 1
4,3593387,9534e3c9-f346-43d5-8400-d1c0e568944e,60835933870012,0.277585,2025-08-02 10:46:24,2025-08-02,2025-08,Train,0,1,SIL Competitor,Appliance,v2,Trench 1


In [63]:
df2 = dfd.copy()

In [64]:
df2['modelVersionId'].value_counts()

modelVersionId
v2    136812
v1    132827
Name: count, dtype: int64

In [65]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287676 entries, 0 to 287675
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             287676 non-null  object        
 1   digitalLoanAccountId   287676 non-null  object        
 2   loanAccountNumber      287676 non-null  object        
 3   Alpha_cic_sil_score    287676 non-null  object        
 4   appln_submit_datetime  287676 non-null  datetime64[us]
 5   disbursementdate       287676 non-null  dbdate        
 6   Application_month      287676 non-null  object        
 7   Data_selection         287676 non-null  object        
 8   deffstpd30             287676 non-null  Int64         
 9   flg_mature_fstpd_30    287676 non-null  Int64         
 10  new_loan_type          287676 non-null  object        
 11  loan_product_type      287676 non-null  object        
 12  modelVersionId         287676 non-null  obje

In [66]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffstpd30', 'FSTPD30')

# gini_results = calculate_periodic_gini_producttype(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffstpd30', 
#     'FSTPD30',
#     product_column='new_loan_type'  
# )

# gini_results = calculate_periodic_gini_prod_ver_trench(
#     df_concat,
#     'Alpha_cic_sil_score', 
#     'deffstpd30', 
#     'FSTPD30',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     product_column='new_loan_type'
# )

In [69]:
f4 = df_concat.copy()
f4

,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3418475,058288be-7950-441f-943c-0652f25ab88b,60834184750015,0.086324,2025-05-04 10:18:56,2025-05-04,2025-05,Test,0,1,SIL ZERO,Appliance,v1,ALL
1,3418539,eb92cc66-336f-4e6b-b42c-828e8425e272,60834185390014,0.096562,2025-05-04 10:29:14,2025-05-04,2025-05,Test,0,1,SIL-Instore,Mall,v1,ALL
2,3418607,dd5efe11-d2ad-43ae-946c-e23dd7b6d36e,60834186070019,0.098974,2025-05-04 10:54:26,2025-05-04,2025-05,Test,0,1,SIL-Instore,Appliance,v1,ALL
3,3418436,996b8104-9db8-45cf-b491-ebe07b97217f,60834184360018,0.049432,2025-05-04 09:38:43,2025-05-04,2025-05,Test,0,1,SIL-Instore,Appliance,v1,ALL
4,3418626,9911a87c-4305-4036-982f-6a52133df3f2,60834186260012,0.179972,2025-05-04 11:06:10,2025-05-04,2025-05,Test,0,1,SIL-Instore,Appliance,v1,ALL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287671,3573980,85b260ec-3f4e-49b0-906a-d72b5a66d21c,60835739800013,0.414477,2025-07-23 16:09:39,2025-07-23,2025-07,Train,0,1,SIL Competitor,Appliance,v2,Trench 1
287672,3581842,5e7e95fe-4cff-4b9b-908a-90a7fd1efce4,60835818420011,0.180635,2025-07-27 13:52:04,2025-07-27,2025-07,Train,0,1,SIL Competitor,Appliance,v2,Trench 1
287673,3581465,43e74354-9b63-46dd-b4b6-c1c6c99f7f46,60835814650016,0.465372,2025-07-27 12:15:39,2025-07-27,2025-07,Train,0,1,SIL-Instore,Appliance,v2,Trench 1
287674,3537891,04aee6dc-7f9e-4b28-87e6-8290065950b2,60835378910019,0.314772,2025-07-04 18:00:45,2025-07-04,2025-07,Train,0,1,SIL-Instore,Appliance,v2,Trench 1


In [71]:
f4['modelVersionId'].value_counts()

modelVersionId
v1    150864
v2    136812
Name: count, dtype: int64

In [72]:
f4.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'Alpha_cic_sil_score', 'appln_submit_datetime', 'disbursementdate',
       'Application_month', 'Data_selection', 'deffstpd30',
       'flg_mature_fstpd_30', 'new_loan_type', 'loan_product_type',
       'modelVersionId', 'trenchCategory'],
      dtype='object')

## combining the dataframe

In [73]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'Alpha_cic_sil_score', 'appln_submit_datetime', 'disbursementdate',
       'Application_month', 'Data_selection','new_loan_type', 'loan_product_type',
       'modelVersionId', 'trenchCategory']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'modelDisplayName', 'Alpha_cic_sil_score', 'appln_submit_datetime',
       'disbursementdate', 'disbursement_month', 'disbmonth',
       'disbursedweek', 'Data_selection', 'deffpd0', 'flg_mature_fpd0',
       'new_loan_type', 'modelVersionId', 'trenchCategory',
       'loan_product_type', 'Application_month', 'deffpd10',
       'flg_mature_fpd10', 'deffpd30', 'flg_mature_fpd30', 'deffspd30',
       'flg_mature_fspd_30', 'deffstpd30', 'flg_mature_fstpd_30'],
      dtype=object)